In [1]:
# Tools for monitoring audio and maybe recording too
%matplotlib inline

import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import os

def make_folder(bird, sess, path_type, exp_folder = '/Volumes/gentner/earneodo/bci_zf/'):
    assert(os.path.isdir(exp_folder))
    bird_folder = os.path.join(os.path.abspath(exp_folder), path_type, bird)
    sess_folder = os.path.join(bird_folder, str(sess).zfill(3))
    if not os.path.isdir(sess_folder):
        os.makedirs(sess_folder)
    return sess_folder

def dated_file_path(path_pars = None):
    if path_pars == None:
        sess_path = os.getcwd()
    else:
        try:
            sess_path = path_pars['sess_path']
        except:
            print 'Wrong path_parameters (sess_path key missing)'
    
    st = time.strptime(time.ctime())
    file_name = "{:04d}{:02d}{:02d}t{:02d}{:02d}{:02d}_sound.wav".format(st.tm_year, st.tm_mon, st.tm_mday, 
                                                                 st.tm_hour, st.tm_min, st.tm_sec)
    return os.path.join(sess_path, file_name)

def unpack_bits(stream, dtype='<h', n_chans=1):
    formatted = np.fromstring(stream, dtype=np.dtype(dtype))
    return formatted

def rms(x):
    return np.linalg.norm(np.array(x, dtype=np.int64))/np.sqrt(x.size)

def mad(x):
    med = np.median(x)
    dev = np.abs(x - np.median(x))
    return np.median(dev)

def msg(string):
    print string
    sys.stdout.flush()

class audioBuffer:
    
    def __init__(self, dtype='<h', n_chans=1):
        self._stream = ''
        self.dtype = '<h'
        self.n_chans = n_chans

    def append(self, new_stream):
        self._stream = self._stream + new_stream
    
    def write(self, new_stream):
        self._stream = new_stream
        
    def read_binary(self):
        return self._stream
    
    # size in bytes
    def get_size(self):
        return len(self._stream)
    
    def get_n_formatted(self):
        return len(self._stream) / np.zeros(1, dtype=np.dtype(self.dtype)).nbytes
    
    def clear_data(self):
        self._stream = ''
    
    def read_formatted(self):
        return unpack_bits(self._stream, self.dtype, self.n_chans)


class Recorder:
    
    
    def __init__(self, channels=1, rate=44100, frames_per_buffer=1024, monitor_pars=None, path_pars=None):
        self.channels = channels
        self.rate = rate
        self.sampling_step_ms = 1000./rate
        self.frames_per_buffer = frames_per_buffer
        self._pa = pyaudio.PyAudio()
        self._stream = None
        self.stream_buffer = audioBuffer(dtype='<h', n_chans = channels)
        self.wavefile = None
    
        self._is_recording = False
        self.rms_thresh = 2000.
        self.rms_stop_thresh = 1400.
        self.monitor_channel = 0
        self.monitor_buffer_size_ms = 2000
        self.monitor_buffer_max_elem = self.monitor_buffer_size_ms/self.sampling_step_ms * channels
        self.monitor_status = 'off'
        self.recorded_samples = 0
        self.record_epoch_ms = 10000 #records maximum 60 sec epochs
        self.record_epoch_max_elem = self.record_epoch_ms/self.sampling_step_ms * channels
        self.ms_in_buf = 0
        self.callback_count = 0
        
        self.path_pars = path_pars
        self.monitor_pars = monitor_pars #TODO: for entering and updating monitor parameters (thresholds, channel, etc)
        
    def msec_to_frames(self, n_msec):
        return np.int(np.ceil(self.rate/(1000.*self.frames_per_buffer)*n_msec))
    
    
    def read_frames(self, n, msec=True):
        self._stream = self._pa.open(format=pyaudio.paInt16,
                                    channels=self.channels,
                                    rate=self.rate,
                                    input=True,
                                    frames_per_buffer=self.frames_per_buffer)
        self._is_recording = True
        n_frames = self.msec_to_frames(n) if msec else n_frames
        self.stream_buffer.clear_data()
        for frame in range(n_frames):
            self.stream_buffer.append(self._stream.read(self.frames_per_buffer))
        self._is_recording = False
        return self.stream_buffer
    
    def get_avg_rms(self, window_len = 3000):
        rms_buffer_formatted = self.read_frames(3000).read_formatted()
        
        return rms(rms_buffer_formatted), np.median(rms_buffer_formatted), mad(rms_buffer_formatted)
    
    
    def start_triggered_mode(self):
        msg("starting monitoring")
        self._stream = self._pa.open(format=pyaudio.paInt16,
                            channels=self.channels,
                            rate=self.rate,
                            input=True,
                            frames_per_buffer=self.frames_per_buffer,
                            stream_callback=self.get_callback())
        
        self.stream_buffer.clear_data()
        self.monitor_status = 'armed'
        self._stream.start_stream()
        return self
    
    def stop_triggered_mode(self):
        msg('Stopping monitor: ' + self.monitor_status)
        if self.monitor_status == 'triggered':
            self.stop_triggered_recording()
            
        self._stream.stop_stream()
        self.monitor_status = 'off'
        
        
    def get_callback(self):
        def callback(in_data, frame_count, time_info, status):
            
            self.callback_count = self.callback_count + 1
            self.stream_buffer.append(in_data)            
            elem_in_buf = self.stream_buffer.get_n_formatted()
            
            # Decide whether to analyze state of the machine
            if elem_in_buf > self.monitor_buffer_max_elem:
                #print "full buffer"
                #plt.plot(self.stream_buffer.read_formatted())
                #msg(str(rms(self.stream_buffer.read_formatted())))
                
                if self.monitor_status == 'armed':
                    msg('Armed with rms ' + str(rms(self.stream_buffer.read_formatted())))
                    
                    if rms(self.stream_buffer.read_formatted()) > self.rms_thresh:
                        print('Should start recording')
                        sys.stdout.flush()
                        self.start_triggered_recording()
                        self.recorded_samples = elem_in_buf

                
                elif self.monitor_status == 'triggered':
                    msg('Triggered with rms ' + str(rms(self.stream_buffer.read_formatted())))
                    if rms(self.stream_buffer.read_formatted()) >= self.rms_stop_thresh:
                        self.continue_triggered_recording()
                        self.recorded_samples = self.recorded_samples + elem_in_buf
                        if self.recorded_samples > self.record_epoch_max_elem:
                            msg('Recording went too long, stopping it')
                            self.stop_triggered_recording()
                    else:
                        self.stop_triggered_recording()
                
                self.stream_buffer.clear_data()
            return in_data, pyaudio.paContinue
        return callback
    
    def start_triggered_recording(self):
        print "Starting to record"
        sys.stdout.flush()
        file_name = self.make_file_path()
        self._prep_file(file_name)
        self._buffer_to_file()
        self.monitor_status = 'triggered'
        
    def continue_triggered_recording(self):
        print "continuing recording"
        sys.stdout.flush()
        self._buffer_to_file()
    
    def stop_triggered_recording(self):
        print "Recording Stopped"
        self._buffer_to_file()
        self._close_file()
        self.monitor_status = 'armed'
        return self
        
    def make_file_path(self):
        return dated_file_path(self.path_pars)
    
    def _prep_file(self, file_name, mode='wb'):
        msg("preparing file " + file_name)
        self.wavefile = wave.open(file_name, mode)
        self.wavefile.setnchannels(self.channels)
        self.wavefile.setsampwidth(self._pa.get_sample_size(pyaudio.paInt16))
        self.wavefile.setframerate(self.rate)
        msg('file open')
    
    def _close_file(self):
        self.wavefile.close()
        
    def _buffer_to_file(self):
        # send the data to file
        print "storing data"
        self.wavefile.writeframes(self.stream_buffer.read_binary())
        #plt.plot(self.stream_buffer.read_formatted())
        self.stream_buffer.clear_data()



## Now do something with all this
exp_folder = '/Volumes/gentner-1/earneodo/bci_zf/'
bird = 'z007'
sess = 1

sess_path = make_folder(bird, sess, 'raw_data', exp_folder=exp_folder)
moni = Recorder(path_pars={'sess_path': sess_path})
#moni.start_triggered_mode()
#bufread = moni.read_frames(1000)

ImportError: No module named pyaudio

In [ ]:
path_pars = {'sess_path' : sess_path}

In [ ]:
dated_file_path(moni.path_pars)

In [ ]:
moni.rms_thresh = 2000

In [2]:
moni.start_triggered_mode()

starting monitoring


<__main__.Recorder instance at 0x107bb5ea8>

Armed with rms 550.83482332
Armed with rms 258.838900947
Armed with rms 75.5198953321
Armed with rms 70.9709306037
Armed with rms 64.3410125379
Armed with rms 90.9137124752
Armed with rms 134.672198032
Armed with rms 120.810176868
Armed with rms 239.720627773
Armed with rms 93.3811686287
Armed with rms 66.441981935
Armed with rms 78.2012033528
Armed with rms 293.183582477
Armed with rms 69.8596426286
Armed with rms 59.8799233617
Armed with rms 59.4696058906
Armed with rms 66.5250819798
Armed with rms 104.063822381
Armed with rms 67.7041177125
Armed with rms 71.3084382375
Armed with rms 76.4020703644
Armed with rms 58.6287994788
Armed with rms 88.4231843969
Armed with rms 58.9745966765


In [3]:
moni.stop_triggered_mode()

Stopping monitor: armed


In [ ]:
bufread = moni.read_frames(2000)
chunk = bufread.read_formatted()
plt.plot(chunk)
print rms(chunk)
print np.std(chunk)
print mad(chunk)

In [ ]:
moni.monitor_status

In [ ]:
moni.stop_triggered_mode()

In [ ]:
moni.stop_triggered_mode()